# Homework 3: Visualizing Data with Movies

Name: Sofia Mohammed

Date: 09/19/2024 

#### Overview
In this homework, you'll create a movie visualization of a model parameter such as temperature, salinity, velocity, or any other field which may be of interest to you. 

As an example, I've provided a movie of circulation in the Equatorial Pacific from the NASA ECCO Version 5 State Estimate. For the purposes of this visualization, I subsetted the Equatorial Pacific is the region between 155-75$^{\circ}$W and $\pm$20$^{\circ}$N.

#### Import Modules
Import the modules required to access data from netCDF files, plot it, and stitch panels together into a movie:

In [ ]:
# import modules here
import os
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
import moviepy.video.io.ImageSequenceClip
import xarray as xr

## Part 1: Download Data
To begin this exercise, download all of the available data for your variable of interest over the timespan of interest.

For the example provided, I've used daily sea surface temperature data for a given year (2015) from the ECCO Version 5 Alpha output [HERE](https://ecco.jpl.nasa.gov/drive/files/Version5/Alpha/latlon_daily/SST.nc).

To obtain your data, use the [Downloading ECCO Data](https://github.com/ProfMikeWood/ocean_modeling_book/blob/main/tools/Downloading%20ECCO%20Data.ipynb) notebook provided with the course notes. Be sure to edit the fields to obtain the correct data.

In [ ]:
# define the path to the data folder
data_folder = '/Users/sofia/Documents/CS185C/Data'

## Part 2: Create a Single Plot of Sea Surface Temperature
Before making a movie, its useful to fine tune a single plot to get the details right. Once you've got your plot lookin' good, you can loop through all of the data to generate the panels of your movie.

Here, make a plot of one variable field. 

Begin by reading in the longitude, latitude, and SST fields from the netCDF file:

In [ ]:
# read in the data
ds = xr.open_dataset(data_folder + '/ECCO/Version5/Alpha/latlon_daily/SST.nc/SST_20150101.nc', decode_times=False)
longitude = np.array(ds['LONGITUDE_T'][:])
latitude = np.array(ds['LATITUDE_T'][:])
SST = np.array(ds['SST'][:])
ds.close()

Next, find the locations of the longitude and latitude bounds within the global grid for your region of interest:

In [ ]:
# define the longitude/latitude bounds
min_lon = -155
max_lon = -75
min_lat = -20
max_lat = 20

# compute the min/max row/col in the data grid
min_row = np.argmin(np.abs(latitude - (min_lat)))
max_row = np.argmin(np.abs(latitude - (max_lat)))
min_col = np.argmin(np.abs(longitude - (min_lon)))
max_col = np.argmin(np.abs(longitude - (max_lon)))

Use the row and column bounds to subset the longitude, latitude, and your variable:

In [ ]:
# subset the data
SST_subset = SST[0,min_row:max_row,min_col:max_col]
lon_subset = longitude[min_col:max_col]
lat_subset = latitude[min_row:max_row]

Finally, make a plot with your subsetted data. For the plot specification, you can use any colormap, data range, and format as you like.

For my example, I find that the temperature plot looks nice with a figure size of (8,4), a turbo colormap, and a data range of 20-31. However, you are encouraged to format your plot to your tastes.

In [ ]:
# make your figure here
# make a figure
fig = plt.figure()
C = plt.pcolormesh(lon_subset, lat_subset, SST_subset, vmin=20, vmax=31) 
# format the axes
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.title('Sea Surface Temperature')
plt.colorbar(C, orientation='vertical',
             label='Temperature (ºC)')

# show the plot
plt.show()

## Part 3: Formulate a Plotting Function
When you are happy with your plot, make your code into a function that will take the year, month, and day as inputs, and output the figure into a given directory. 

In [ ]:
frames_directory = '/Users/sofia/Documents/CS185C/Homework/Homework_3/frames_2'
panel_list = []
def plot_panel(year,month,day,):
    # fill in your function here
    data_folder = ('/Users/sofia/Documents/CS185C/Data')
    str_year = str(year)
    data_file = 'SST_'+str_year+'{:02d}'.format(month)+'{:02d}'.format(day)+'.nc'
    ds = xr.open_dataset(data_folder+'/ECCO/Version5/Alpha/latlon_daily/SST.nc/'+data_file, decode_times=False)
    longitude = np.array(ds['LONGITUDE_T'][:])
    latitude = np.array(ds['LATITUDE_T'][:])
    SST = np.array(ds['SST'][:])
    ds.close()

    SST_subset = SST[0,min_row:max_row,min_col:max_col]
    lon_subset = longitude[min_col:max_col]
    lat_subset = latitude[min_row:max_row]
    fig = plt.figure()
    C = plt.pcolormesh(lon_subset, lat_subset, SST_subset, vmin=20, vmax=31) 
    # format the axes
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.title('Sea Surface Temperature')
    plt.colorbar(C, orientation='vertical',
                 label='Temperature (ºC)')
    
    # show the plot
    image_file = os.path.join(frames_directory,
                             'SST_'+str_year+'{:02d}'.format(month)+'{:02d}'.format(day)+'.png')
    plt.savefig(image_file)
    plt.close(fig)
    panel_list.append(image_file)

Once the function is complete, use it to loop through all of the data to make a plot for each day of the year. The following lines of code will form the panels of a data visualization movie when using daily data. If you are using monthly data, update accordingly:

In [ ]:

# loop through the data and make a plot for each day
year = 2015
for month in range(1,13):
    if month in [1,3,5,7,8,10,12]:
        n_days = 31
    elif month in [4,6,9,11]:
        n_days = 30
    else:
        if year%4==0:
            n_days = 29
        else:
            n_days = 28
    for day in range(1,n_days+1):
        plot_panel(year,month,day)
        

## Part 4: Compile Panels into a Movie
After creating all of the panels, stitch them together into a movie using moviepy:

In [ ]:
# make a list of files for each movie panel
# sort the panels

In [ ]:
# set the frames per second
fps=5

# use the ImageSequenceClip module to set up the clip
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(panel_list, fps=fps)

# write the video to a file
clip.write_videofile('Sea_Surface_Temperature.mp4')

## Part 5: Interpret Features in the Movie
A movie gives a good visualization of the physical processes occurring in a model. Describe one feature you see in the movie and relate it to an oceanographic feature. It could be a feature discussed in class or another feature you noticed and researched on your own.

Answer: The movie conveyed information about the pattern of temperature rising and falling throught the ocean at the given cooridinates. The region that was selected, spesifically shows parts of the pacific ocean both above and below the equator. The section of the equator that is contained with in the chosen plot, is distnct due the its straight path of warmer temperatures. Along with this, paths of water cooler temperatures can be seen moving along the coastline, both above and below the equator. Due to their circular motions, I believe they may be related to ocean gyres. The cooler temperatures that are slowly emerging at the bottom right corner of the plots could be due to currents moving cold water, near the southern pole, further north.